## Example using GAM_igraph


In [1]:
import igraph as ig
import ecg
import GAM_igraph
import numpy as np


In [2]:
## build graph with n=1000 vertices and n_comm=10 communities
## p_in : edge probability within a community
## p_out: edge probability between communities
n_comm = 10
p_in = .1
p_out = 0.025
P = np.full((n_comm,n_comm),p_out)
np.fill_diagonal(P,p_in)
## ground truth communities is stored in 'class' attribute
g = ig.Graph.Preference(n=1000, type_dist=[1/n_comm]*n_comm, pref_matrix=P.tolist(), attribute='class')
gt = ig.clustering.VertexClustering.FromAttribute(g,'class')

In [3]:
## run Louvain and ECG:
ml = g.community_multilevel()
ec = g.community_ecg(ens_size=32)

## compare a few Graph-Aware Measures (GAM)
print('Adjusted Graph-Aware Rand Index for Louvain:',g.GAM(ml,gt))
print('Adjusted Graph-Aware Rand Index for ECG:',g.GAM(ec,gt))
print('\nJaccard Graph-Aware for Louvain:',g.GAM(ml,gt,method="jaccard",adjusted=False))
print('Jaccard Graph-Aware for ECG:',g.GAM(ec,gt,method="jaccard",adjusted=False))


Adjusted Graph-Aware Rand Index for Louvain: 0.41333267708205457
Adjusted Graph-Aware Rand Index for ECG: 0.8568919005508913

Jaccard Graph-Aware for Louvain: 0.4229779937124893
Jaccard Graph-Aware for ECG: 0.8201058201058201


## Example using GAM_networkx

In [4]:
import networkx as nx
import GAM_networkx
import numpy as np


In [5]:
# Graph generation with 10 communities of size 10
commSize = 10
numComm = 10
G = nx.generators.planted_partition_graph(l=numComm,k=commSize,p_in=0.3,p_out=0.025)
true_comm = [set(list(range(commSize*i, commSize*(i+1)))) for i in range(numComm)]

In [6]:
# Not many partitionning algorithms in networkx ...
# Here using Girvan_Newman with the right number of communities:
algo_comm = list(nx.algorithms.community.girvan_newman(G))[numComm-2]

In [7]:
print("Adjusted Graph-Aware Rand Index for Girvan-Newman:",G.GAM(true_comm, algo_comm))
print("Jaccard Graph-Aware for Girvan-Newman:",G.GAM(true_comm, algo_comm, method="jaccard",adjusted=False))


Adjusted Graph-Aware Rand Index for Girvan-Newman: 0.1927746520580396
Jaccard Graph-Aware for Girvan-Newman: 0.46285714285714286


In [8]:
## adjusted RAND index requires a different format (label for each vertex)
from sklearn.metrics import adjusted_rand_score as ARI

## build dictionaries for the communities
tc = {v:idx for idx, part in enumerate(true_comm) for v in part}
ac = {v:idx for idx, part in enumerate(algo_comm) for v in part}

## compute ARI
print("Adjusted non-Graph-Aware Rand Index for Girvan-Newman:",ARI(list(tc.values()), list(ac.values())))

## dictionaries can also be used with GAM
print("Adjusted Graph-Aware Rand Index for Girvan-Newman:",G.GAM(tc, ac))

Adjusted non-Graph-Aware Rand Index for Girvan-Newman: 0.4970906068162926
Adjusted Graph-Aware Rand Index for Girvan-Newman: 0.1927746520580396
